# Importing Libraries

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

ImportError: cannot import name 'to_categorical' from 'keras.utils' (C:\Users\shiva\anaconda3\lib\site-packages\keras\utils\__init__.py)

In [3]:
data=[]
labels=[]

height = 30
width = 30
channels = 3
classes = 43
n_inputs = height * width*channels

In [4]:
for i in range(classes) :
    path = "train/{0}/".format(i)
    print(path)
    Class=os.listdir(path)
    for a in Class:
        try:
            image=cv2.imread(path+a)
            image_from_array = Image.fromarray(image, 'RGB')
            size_image = image_from_array.resize((height, width))
            data.append(np.array(size_image))
            labels.append(i)
        except AttributeError:
            print(" ")
            
Cells=np.array(data)
labels=np.array(labels)

train/0/


NameError: name 'cv2' is not defined

# Randomize the order of the input images

In [ ]:
s=np.arange(Cells.shape[0])
np.random.seed(43)
np.random.shuffle(s)
Cells=Cells[s]
labels=labels[s]

# Splitting training and testing dataset

In [ ]:
(X_train,X_val)=Cells[(int)(0.2*len(labels)):],Cells[:(int)(0.2*len(labels))]
X_train = X_train.astype('float32')/255 
X_val = X_val.astype('float32')/255
(y_train,y_val)=labels[(int)(0.2*len(labels)):],labels[:(int)(0.2*len(labels))]



# Converting the labels into one hot encoding

In [ ]:

y_train = to_categorical(y_train, 43)
y_val = to_categorical(y_val, 43)

# Building Model

In [ ]:
cnn = tf.keras.models.Sequential()

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(5,5), activation='relu',))

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [ ]:
cnn.add(tf.keras.layers.Dropout(rate =0.25))

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [ ]:
cnn.add(tf.keras.layers.Dropout(rate =0.25))

In [ ]:
cnn.add(tf.keras.layers.Flatten())

In [ ]:
cnn.add(tf.keras.layers.Dense(units=256, activation='relu'))

In [ ]:
cnn.add(tf.keras.layers.Dropout(rate =0.5))

In [ ]:
cnn.add(tf.keras.layers.Dense(units=43, activation='softmax'))

# Compiling the Model

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Training the Models and Saving the accuracy for each epochs

In [ ]:
Model = cnn.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_val, y_val))

# Plotting graphs for accuracy

In [ ]:
plt.figure(0)
plt.plot(Model.history['accuracy'], label='training accuracy')
plt.plot(Model.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show() 

plt.figure(1)
plt.plot( Model.history['loss'], label='training loss')
plt.plot(Model.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

# Testing accuracy on test dataset

In [ ]:
from sklearn.metrics import accuracy_score
y_test = pd.read_csv('Test.csv')
labels = y_test["Path"].values
y_test = y_test["ClassId"].values
data=[]
for f in labels:
    image=cv2.imread('test/'+f.replace('Test/', ''))
    image_from_array = Image.fromarray(image, 'RGB')
    size_image = image_from_array.resize((height, width))
    data.append(np.array(size_image))

X_test=np.array(data)
X_test = X_test.astype('float32')/255 
pred = cnn.predict_classes(X_test)

# Accuracy with the test data

In [ ]:
print(y_test)

In [ ]:
print(pred)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, pred))

# Saving the Model For Tkinter GUI

In [ ]:
cnn.save('Traffic Sign Classifier.h5')